In [1]:
from models import EfficientNet3D
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [2]:
num_classes = 3
model = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': num_classes}, in_channels=1)

Block args [BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16, expand_ratio=1, id_skip=True, stride=[2], se_ratio=0.25)]
Block args [BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16, expand_ratio=1, id_skip=True, stride=[2], se_ratio=0.25), BlockArgs(kernel_size=3, num_repeat=2, input_filters=16, output_filters=24, expand_ratio=6, id_skip=True, stride=[1], se_ratio=0.25)]
Block args [BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16, expand_ratio=1, id_skip=True, stride=[2], se_ratio=0.25), BlockArgs(kernel_size=3, num_repeat=2, input_filters=16, output_filters=24, expand_ratio=6, id_skip=True, stride=[1], se_ratio=0.25), BlockArgs(kernel_size=5, num_repeat=2, input_filters=24, output_filters=40, expand_ratio=6, id_skip=True, stride=[1], se_ratio=0.25)]
Block args [BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16, expand_ratio=1, id_skip=True, stride=[2], se_ratio=0.25), BlockArgs(kern

In [3]:
model = model.to(device)
inputs = torch.randn((1, 45, 15, 15, 15)).to(device)
labels = torch.tensor([0]).to(device)
# test forward

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model.train()

EfficientNet3D(
  (_conv_stem): Conv3dStaticSamePadding(
    1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1, 0, 1))
  )
  (_bn0): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock3D(
      (_depthwise_conv): Conv3dStaticSamePadding(
        32, 32, kernel_size=(3, 3, 3), stride=[2, 2, 2], groups=32, bias=False
        (static_padding): ZeroPad2d((0, 1, 0, 1, 0, 1))
      )
      (_bn1): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv3dStaticSamePadding(
        32, 8, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv3dStaticSamePadding(
        8, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv3dStaticSamePadding(
        32, 16, kernel_siz

In [4]:
count = sum(p.numel() for p in model.parameters() if p.requires_grad)
count

4692223

In [5]:
for epoch in range(1):
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    print('[%d] loss: %.3f' % (epoch + 1, loss.item()))

print('Finished Training')
print(outputs.shape)
print(outputs.cpu().detach().numpy())

Shape of x in static padding torch.Size([45, 1, 16, 16, 16])
Conv3D in padding shape torch.Size([45, 32, 7, 7, 7])
Block 0 x shape torch.Size([45, 32, 7, 7, 7])
Shape of x in static padding torch.Size([45, 32, 8, 8, 8])
Conv3D in padding shape torch.Size([45, 32, 3, 3, 3])
Swish after depthwise x shape torch.Size([45, 32, 3, 3, 3])
Squeeze shape torch.Size([45, 32, 3, 3, 3])
Shape of x in static padding torch.Size([45, 32, 1, 1, 1])
Conv3D in padding shape torch.Size([45, 8, 1, 1, 1])
Shape of x in static padding torch.Size([45, 8, 1, 1, 1])
Conv3D in padding shape torch.Size([45, 32, 1, 1, 1])
Excite shape torch.Size([45, 32, 3, 3, 3])
Sigmoid and x shape torch.Size([45, 32, 3, 3, 3])
Shape of x in static padding torch.Size([45, 32, 3, 3, 3])
Conv3D in padding shape torch.Size([45, 16, 3, 3, 3])
Project conv shape torch.Size([45, 16, 3, 3, 3])
Block 1 x shape torch.Size([45, 16, 3, 3, 3])
Shape of x in static padding torch.Size([45, 16, 3, 3, 3])
Conv3D in padding shape torch.Size([45